### Bipolar Montages  
  
This notebook will generate EEG tracings and plot those tracings to compare between and within activity types.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fastparquet, pyarrow
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('working_data.csv')

##### Splitting Data by Activity Type

In [3]:
def activity_df(activity_type):
    return df[df['expert_consensus'] == activity_type].reset_index().drop(columns = 'index')

seizure_df = activity_df('Seizure')
lpd_df = activity_df('LPD')
gpd_df = activity_df('GPD')
lrda_df = activity_df('LRDA')
grda_df = activity_df('GRDA')
other_df = activity_df('Other')

##### Functions to Scale and Plot EEG Signals

In [4]:
def scale(df, row):
    eeg = pd.read_parquet('train_eegs/{}.parquet'.format(df['eeg_id'][row]), engine = 'pyarrow')
    start = int(df['eeg_label_offset_seconds'][row] * 200)
    finish = start + 10000
    sub_eeg = eeg[start: finish]
    sub_eeg = sub_eeg.reset_index().drop(columns = 'index')
    electrodes = sub_eeg.columns
    scaler = StandardScaler()
    sub_eeg = scaler.fit_transform(sub_eeg)
    return pd.DataFrame(sub_eeg, columns = electrodes)

def activity_signal_plots(sub_eegs, activities, electrodes, h, w):
    fig, axs = plt.subplots(len(electrodes), len(sub_eegs))
    fig.set_figheight(h)
    fig.set_figwidth(w)
    fig.tight_layout()
    for i in range(len(electrodes)):
        if len(sub_eegs) == 1:
            axs[i].plot(sub_eegs[0][electrodes[i]])
            axs[i].set_title('{}_{}'.format(activities[0], electrodes[i]))
        else:
            for j in range(len(sub_eegs)):
                axs[i][j].plot(sub_eegs[j][electrodes[i]])
                if len(activities) > 1:
                    axs[i][j].set_title('{}_{}'.format(activities[j], electrodes[i]))
                else:
                    axs[i][j].set_title('{}_{}'.format(activities[0], electrodes[i]))
    plt.show()

### Double Banana  
  
Right Parasagittal: Fp2, F4, C4, P4, O2  
Right Lateral: Fp2, F8, T4, T6, O2  
Left Parasagittal: Fp1, F3, C3, P3, O2  
Left Lateral: Fp1, F7, T3, T5, O2  
Midline: Fz, Cz, Pz  
  
Channels are pairs within each group and tracings are generated by subtracting the second electrode in a pair from the first.  
  
Below I will right a function which generates a new sub EEG from existing sub EEGs with EEG tracings for columns instead of single electrode signals. I will then take a few sub EEGs at random from each activity type, scale the sub EEG, generate its tracings, and then plot those tracings.  
  
Double Banana isn't the only montage. I will being generating tracings with other montages as well. That will likely be done in separate notebooks.